# Flows and cuts

A flow graph is a directed graph $G=(V,E)$ whose edges are quantified in capacity for flow. In the adjacency matrix representation, $G[i][j]$ is the capacity of the edge from vertex $i$ to vertex $j$. If there is no edge $i→j$, the capacity is 0 and correspondingly, $G_[i][j]=0$. A flow is anything that can be measured in time units. Bits per second in a network, cars per hour in a highway, etc. The capacity of an edge is the maximum flow that can traverse the edge.

In a flow graph, typically we are interested in finding the maximum flow between two vertices that we label _source_ and _target._ These can be any two vertices in the graph.

The **Ford-Fulkerson** method finds the maximum flow that can be pushed through a network, between a source and a target vertex, as follows:

```text
initialize max_flow to 0
while there is an augmenting path:
  add augmenting path's min residual capacity to max_flow
return max_flow
```

The technique, as stated above, depends on _augmenting paths_. As long as we figure out what an augmenting path is, we are all set.


## Notation

Instead of a weight, edges in flow graphs have a capacity. We write $u \xrightarrow{10} v$ to show that the capacity of the edge from vertex $u$ to $v$ is 10.

Capacities, and flows, are typically expressed in unit time such as seconds, hours, etc. For example, a road may have a capacity of 10 cars per minute, an wire has a capacity of $6.24\times 10^8$ electrons per second, and a grocery register has a capacity of 20 clients per hour. In a flow graph, we do not show explicitly the units. We refer to the capacity just by its value, e.g., a "capacity of 10". The assumption is that we are in agreement of what that 10 means in the context of the graph we study (could be 10 cars a minute, 20 clients per hour, etc).

When there is flow across an edge, we show its value too, with a second number: $u \xrightarrow{5/10} v$ indicates that there is a flow of 5 (something per some unit time) across an edge that can accommodate up to 10 somethings per unit time.

If we wish to be dramatic, we could write $u \xrightarrow{0/10} v$ to emphasize that there is no flow over an edge. In desperate situations, we may even write $u \xrightarrow{0} v$ to indicate an edge with 0 capacity; though it may be best if we just admitted there is no edge between $u$ and $v$. Algorists that write $u \xrightarrow{0/0} v$ to indicate there is no flow over a non-existing edge, should not be allowed near computers.

## Alternative notation

Instead of writing $u \xrightarrow{3/10} v$ to show that there is a flow of 3 over an edge with capacity 10, we can use two edges that represent effective capacities: a forward edge $u\rightarrow v$ whose capacity is now 7 (since there is a flow of 3 in that direction) and a backward edge $u \leftarrow v$ whose capacity is 3.

$$
u \xrightarrow{3/10} v\ \ \textsf{same as:}\  \begin{cases}
  u \xrightarrow{7} v \\
  u \xleftarrow{3} v
\end{cases}
$$

This notation makes it possible to indicate flows in an adjacency matrix in a simple manner. For example, the flow $u \xrightarrow{3/10} v$ in an adjacency matrix $\texttt M$ can be shown, using the decomposition above, as

$$
\begin{align}
\texttt M[u][v] &= 10-3 \\
\texttt M[v][u] & = 3
\end{align}
$$

Without this notation, we would need to store _pairs_ of values in the adjacency matrix, such

$$
\texttt M[u][v] = (3, 10)
$$

requiring a bit more complicated operations at the implementation level. For example the remaining (available) capacity of the edge above can be computed as

$$
\texttt M[u][v][1] - \texttt M[u][v][0]
$$

where the third index points to the element of the pair.


## Augmenting path

An augmenting path is a path from the source vertex to the target vertex, $s\rightsquigarrow t$, in the _residual graph._

## Residual graph

The residual graph comprises the vertices of the input graph connected with edges showing _residual capacities._

## Residual capacity

Given a graph $G=(V,E)$ represented as adjacency matrix $\texttt G[i][j]$ such that $\texttt G[i][j]=0$ when there is no edge $i\rightarrow j$ and $\texttt G[i][j] > 0$ when an edge exists, the residual graph is initialized as $\texttt R=\texttt G$.

Through successive iterations -- as we'll see how they are done -- the edges in the residual graph change until the target vertex is no longer reachable from the source vertex.


## Ford-Fulkerson method

The Ford-Fulkerson method finds the maximum flow possible between two specific vertices in a directed graph. Furthermore, it helps us find the smallest possible set of edges in the graph that, when eliminated, will disrupt the flow between the two specific vertices. The method has several parts, requiring different algorithms.

### Input and Output

The input to the Ford-Fulkerson method is a graph and two of its vertices. The output comprises a value for the maximum flow between the two vertices and a set of edges that will disrupt that flow.

```python
def ford_fulkerson(graph, source, target)
```

The `graph` is given as an adjency matrix of size $n$ and $0 \leq \texttt{source} \neq \texttt{target} < n$.

### Initialization

Most of the word in the Ford-Fulkerson method is done on a copy of the input graph, called the residual graph. Initially, the residulal graph is an exact copy of the input graph.

```python
import copy
residual = copy.deepcopy(graph)
```

We need an accumulator for the maximum flow of the graph

```python
max_flow = 0
```

Finally we need an augmenting path. This is path from source to target vertices in the residual graph -- any path.

A depth-first search is a good technique to use for finding a path. Here's the sketch for a DSF-based method.


In [ ]:
# 12345678901234567890123456789012345678901234567890123456789012345678901234567890
def find_path(graph: list[list[int]], source: int, target: int):
    """Return a path -- any path -- from source to target in the graph"""

    # Initialize return item
    path: list[int] = None

    # Make sure inputs are ok
    if graph is not None:
        n: int = len(graph)
        if n > 0 and (0 <= source < n) and (0 <= target < n):

            # Initialize DFS tools
            no_edge: int = graph[0][0]  # absence of edge
            marked: list[int] = [source]  # vertices already processed
            found: bool = False  # Flags detection of path

            # What vertex to explore next and what is the path
            # to it. The information is stored as a tuple in
            # the form:
            #  (vertex, path_to_this_vertex)
            # with path_to_this_vertex being a list of the
            # vertices alonγ the path.
            stack: list[(int, list[int])] = [(source, [source])]

            while len(stack) > 0 and not found:
                # Explore the next vertex from the stack
                (u, path_from_source_to_u) = stack.pop()
                found = u == target
                if found:
                    # u is the end of the path, so we got what we are
                    # looking for
                    path = path_from_source_to_u
                else:
                    # Explore the neighbors of u, hopefully one of them
                    # will get us a stop closer to the target vertex.
                    v: int = n - 1
                    while v >= 0:
                        if graph[u][v] != no_edge and v not in marked:
                            marked.append(v)
                            stack.append((v, path_from_source_to_u + [v]))
                        v -= 1
    return path

### The principal loop in Ford-Fulkerson

The Ford-Fulkerson technique focuses on the residual graph. Its purpose is to saturatre every path from source to target vertex in the residual graph -- such path is called an augmenting path. The principal loop of the technique is based on the following pseudocode:

$$
\begin{align}
& \textbf{while}\ \text{there is an augmenting path} \\
& \qquad m \leftarrow \text{capacity of smallest edge along the augmenting path} \\
& \qquad \texttt{max\_flow} \leftarrow  \texttt{max\_flow} +  m\quad \texttt{\#}\ \textsf{use alternative notation} \\
& \qquad \text{add}\ m\ \text{flow to every edge along the augmenting path}
\end{align}
$$

At the end of the loop, we have two results: the value of `max_flow` which is the most flow we can send from source to target, and a residual graph with two components -- one comprising the vertices that can be reached from the source vertex and one comprising those that cannot be reached from the source vertex.


### Finding the minimum cut

After the principal loop above ends, the residual graph is split into two components. Let's say that component $\mathcal S$ comprises the services reachable from the source vertex and $\mathcal T$ the vertices not reachable from source (correspondingly, the target vertex is reachable from every vertex in $\mathcal T$ but not from $\mathcal S$.)

This is where things get confusing. We want to find all the edges between vertices in $\mathcal S$ and \mathcal T$ but in the input graph not the residual graph.

The residual graph $R$ is initialy an exact copy of the input graph $R=G=(V,E)$. The principal loop of the Ford-Fulkerson method alters its edges and the residual graph becomes $R=(V,\mathcal E)$, with $\mathcal E\neq E$.

So the problem becomes to find the set $\mathcal C$ of all edges $u\rightarrow v$ such that

$$
u\rightarrow v \in E,\ \text{and}\ u\in \mathcal S,\ \text{and}\ v \in \mathcal T
$$

In other words, we need the edges from the input graph ($G$) whose vertices are in opposite components of $R$. Here's the pseudocode:

$$
\begin{align*}
& \mathcal C \leftarrow \varnothing \\
& \textbf{for}\ u \in [0,n) \\
& \qquad \textbf{for}\ v \in [0,n) \\
& \qquad\qquad \textbf{if}\ G[u][v] \neq G[0][0]\ \textbf{and}\ u\in\mathcal S\ \textbf{and}\ v\in\mathcal T \\
& \qquad\qquad\qquad \mathcal C \leftarrow \mathcal C \cup \{u\rightarrow v\}
\end{align*}
$$

The set $\mathcal C$ is called the minimum cut of $G$ and it's the smallest set of edges we need to eliminate in order to distrupt the flow from source to target. The total weight of its edges equals to the maximum flow we can send from source to target in the input graph, i.e.,

$$
\sum_{u\rightarrow v\ \in\mathcal C} G[u][v] = \texttt{max\_flow}
$$


## Putting it together

Implement the Ford-Fulkerson method, writing as many auxiliary methods as you need.


In [ ]:
# Test graph

G = [  #  A   B   C   D   E
    [0, 20, 0, 0, 0],  # A
    [0, 0, 5, 6, 0],  # B
    [0, 0, 0, 3, 7],  # C
    [0, 0, 0, 0, 8],  # D
    [0, 0, 0, 0, 0],  # E
]

In [ ]:
import copy  # for deepcopy of lists


def ford_fulkerson(graph: [list[list[int]]], source: int, target: int):
    """Computes max flow between source and target vertex and determines the
    minimum cut of the graph."""

    # Guard statements -aka sanity check
    if (
        graph is None
        or len(graph) == 0
        or 0 <= source < len(graph)
        or 0 <= target < len(graph)
    ):
        raise ValueError("Graph is empty or source or target vertex out of range.")

    # Shortcuts and initial values
    n: int = len(graph)  # number of vertices
    max_flow = 0
    min_cut = []
    no_edge = graph[0][0]

    # Initialize the residual graph
    residual = copy.deepcopy(graph)

    # Find a source -> target path in the residual graph
    augmenting_path = find_path(residual, source, target)

    # principal loop
    while augmenting_path is not None and len(augmenting_path) > 1:

        # find the smallest edge in the augmenting path
        u = augmenting_path[0]
        v = augmenting_path[1]
        smallest_edge = residual[u][v]
        for k in range(1, len(augmenting_path) - 1):
            u = augmenting_path[k]
            v = augmenting_path[k + 1]
            if residual[u][v] < smallest_edge:
                smallest_edge = residual[u][v]

        # Add the capacity of the smallest edge to the max flow of the graph
        max_flow += smallest_edge

        # Saturate the smallest edge of this resisual path
        for k in range(len(augmenting_path) - 1):
            u = augmenting_path[k]
            v = augmenting_path[k + 1]
            residual[u][v] -= smallest_edge
            # Create a capacity in the opposite direction
            residual[v][u] += smallest_edge

        # Find another augmenting path - if none, the principal
        # loop will end
        augmenting_path = find_path(residual, source, target)

    # End of principal loop. At this point there are no paths between
    # source and target vertices in the residual graph. And we have
    # computed the max flow of the input graph. Next we have to compute
    # the edges that were severed in the process of eliminating every
    # augmenting path in the residual graph. This is down in two steps.
    # First find the vertices in the two components of the residual
    # graph: one component comprises all the vertices reachable from the
    # course vertex and the other all the vertices not reachable from
    # the source vertex. Then find all the edges that existed between
    # these two component.

    # For simplicity we can mark each vertex in the residual graph
    # as reachable or not from the source -- assume none is reachable
    # to begin with.
    source_reachable = [False] * n

    # Perform a reachability search from source, in the residual graph,
    # marking vertices accordingly
    bag = [source]
    while len(bag) > 0:
        u = bag.pop()
        if u not in source_reachable:
            source_reachable.append(u)
            for v in range(n):
                if residual[u][v] != no_edge:
                    bag.append(v)

    # Now let's explore every edge in the input graph and test
    # if its vertices are in opposite components of the
    # residual graph. Two vertices u and v are in opposite 
    # components of the residual graph if the XOR (^) of their
    # source_reachable values is true, i.e., either u is not
    # reachable from source but v is, or u can be reached from
    # the source and v cannot.
    for u in range(n):
        for v in range(n):
            if graph[u][v] != no_edge and (source_reachable[u] ^ source_reachable[v]):
                min_cut.append([u, v])

    # Done
    return max_flow, min_cut